In [1]:
%load_ext autoreload
%autoreload 1
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import indicators
from load import load

In [4]:
df = load()
df.head()

,close,high,low,open,volume
2017-02-16 00:00:00,1015.57001,1015.57001,1011.600000,1011.60000,356.334534
2017-02-16 00:05:00,1016.00000,1016.00000,1014.835898,1015.57001,7848.530146
2017-02-16 00:10:00,1015.10000,1016.00000,1011.610001,1016.00000,966.125860
2017-02-16 00:15:00,1017.20758,1017.20758,1015.876661,1016.00000,3088.018045
2017-02-16 00:20:00,1015.00000,1017.20758,1015.000000,1017.20758,17.594961


In [5]:
df.describe()

,close,high,low,open,volume
count,289.000000,289.000000,289.000000,289.000000,289.000000
mean,1029.633530,1030.836066,1028.348391,1029.719112,3365.962641
std,7.934833,7.668024,7.949878,7.654546,5621.900901
min,1011.700004,1014.343666,1011.600000,1011.600000,0.000000
25%,1024.100000,1026.147770,1023.365971,1024.819990,354.891554
50%,1028.000000,1029.000000,1027.000003,1027.999997,1323.341255
75%,1037.125678,1038.571130,1036.000000,1037.110111,3573.907939
max,1044.890000,1045.000000,1044.666995,1044.666995,40822.592012


In [6]:
df['y'] = (1 - df['close'] / df['close'].shift(1)).shift(-1)

In [7]:
df = indicators.RSI(df, 14)
df = indicators.STOCHASTICS(df, 14, 3)
df = indicators.ADX(df, 14)

In [8]:
df = df.dropna()

In [9]:
train_data = df[:int(len(df)*0.8)]
test_data = df[int(len(df)*0.8):]

In [14]:
lr = LinearRegression()

In [16]:
lr.fit(train_data[['rsi_14', 'stoch_d_14_3', 'adx_14']].values.reshape(len(train_data), 3), train_data['y'].values)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [18]:
predicted = lr.predict(test_data[['rsi_14', 'stoch_d_14_3', 'adx_14']].values.reshape(len(test_data), 3))

In [19]:
rmse = np.sqrt(mean_squared_error(test_data['y'].values, predicted))
rmse

0.0017937446234165375